Krok 1: Instalacja i Importowanie Bibliotek

In [ ]:
# Instalacja potrzebnych bibliotek
!pip install gspread pandas openpyxl google-auth google-auth-oauthlib google-auth-httplib2

# Importowanie bibliotek
import gspread
import pandas as pd
from google.colab import auth
from google.auth import default

# Autoryzacja i inicjalizacja klienta
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)



Krok 2: Odczytanie Danych z Google Sheets


In [ ]:
# Otwieranie arkusza Google Sheets
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1nJqelCU3lzN1hLqEtxFawqYVfJM7Yvo5lxg1KN1kqi8/edit#gid=1963505034' # Podmień na URL Twojego arkusza
workbook = gc.open_by_url(spreadsheet_url)

# Odczytywanie danych z arkuszy "Oferta" i "Kosztorys"
sheet_oferta = workbook.worksheet('Oferta')
sheet_kosztorys = workbook.worksheet('Kosztorys')

# Konwersja danych na DataFrame
df_oferta = pd.DataFrame(sheet_oferta.get_all_records())
df_kosztorys = pd.DataFrame(sheet_kosztorys.get_all_records())

# Wyczyszczenie danych, jeśli jest taka potrzeba
# Przykład: df_oferta['Opis'] = df_oferta['Opis'].str.strip()


Krok 3: Przypisywanie Cen


In [ ]:
def najlepsze_dopasowanie(opis, df_oferta):
    max_wspolne = 0
    najlepsza_cena = None
    opis_slowa = set(opis.split())

    for _, wiersz in df_oferta.iterrows():
        oferta_slowa = set(wiersz['Material'].split())
        wspolne_slowa = len(opis_slowa.intersection(oferta_slowa))

        if wspolne_slowa > max_wspolne:
            max_wspolne = wspolne_slowa
            najlepsza_cena = wiersz['Cena']

    return najlepsza_cena

# Przypisywanie cen do kosztorysu na podstawie najlepszego dopasowania
df_kosztorys['Cena2'] = df_kosztorys['Opis'].apply(lambda opis: najlepsze_dopasowanie(opis, df_oferta))



Krok 4: Zapis do Nowego Pliku


In [ ]:
# Zapisanie zmodyfikowanego DataFrame do nowego arkusza Google Sheets lub pliku Excel
# Do pliku Excel
df_kosztorys.to_excel('wyniki_kosztorys.xlsx', index=False)

# Jeśli chcesz zapisywać wyniki bezpośrednio do Google Sheets, potrzebujesz dodatkowej konfiguracji i użycia gspread
